In [1]:
import tensorflow as tf
print(tf.__version__)




2.19.0


In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# Load the dataset
df = pd.read_csv("fer2013_combined.csv")

# Convert pixels column to NumPy arrays
X = np.array([np.fromstring(pixels, dtype=np.uint8, sep=' ').reshape(48, 48, 1) for pixels in df["pixels"]])

# Convert labels to categorical (one-hot encoding)
y = to_categorical(df["emotion"], num_classes=7)

# Normalize pixel values (scale between 0 and 1)
X = X / 255.0

# Split into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Define CNN model
model = Sequential([
    Conv2D(64, (3,3), activation='relu', input_shape=(48,48,1)),
    MaxPooling2D(pool_size=(2,2)),
    
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(7, activation='softmax')  # 7 output classes (angry, disgust, fear, etc.)
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=20, batch_size=64)

# Save the model
model.save("emotion_detection_model.h5")

# Evaluate on validation set
val_loss, val_acc = model.evaluate(X_val, y_val)
print(f"Validation Accuracy: {val_acc * 100:.2f}%")


C:\Users\HP\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
449/449 ━━━━━━━━━━━━━━━━━━━━ 79s 165ms/step - accuracy: 0.2928 - loss: 1.7601 - val_accuracy: 0.4223 - val_loss: 1.5227
Epoch 2/20
449/449 ━━━━━━━━━━━━━━━━━━━━ 70s 156ms/step - accuracy: 0.4110 - loss: 1.5113 - val_accuracy: 0.4646 - val_loss: 1.4030
Epoch 3/20
449/449 ━━━━━━━━━━━━━━━━━━━━ 65s 144ms/step - accuracy: 0.4588 - loss: 1.4099 - val_accuracy: 0.4891 - val_loss: 1.3549
Epoch 4/20
449/449 ━━━━━━━━━━━━━━━━━━━━ 66s 146ms/step - accuracy: 0.4812 - loss: 1.3406 - val_accuracy: 0.5095 - val_loss: 1.3068
Epoch 5/20
449/449 ━━━━━━━━━━━━━━━━━━━━ 67s 148ms/step - accuracy: 0.5083 - loss: 1.2792 - val_accuracy: 0.5098 - val_loss: 1.2763
Epoch 6/20
449/449 ━━━━━━━━━━━━━━━━━━━━ 66s 147ms/step - accuracy: 0.5329 - loss: 1.2274 - val_accuracy: 0.5209 - val_loss: 1.2565
Epoch 7/20
449/449 ━━━━━━━━━━━━━━━━━━━━ 62s 138ms/step - accuracy: 0.5465 - loss: 1.1764 - val_accuracy: 0.5245 - val_loss: 1.2405
Epoch 8/20
449/449 ━━━━━━━━━━━━━━━━━━━━ 67s 148ms/step - accuracy: 0.5668 - loss: 1

225/225 ━━━━━━━━━━━━━━━━━━━━ 5s 20ms/step - accuracy: 0.5469 - loss: 1.5085
Validation Accuracy: 54.53%


In [6]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam

# Load the trained model
model = load_model("emotion_detection_model.h5")

# Recompile the model (optional but recommended to avoid the warning)
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Define emotion labels (to map the predictions to human-readable emotions)
EMOTION_LABELS = {
    0: "Angry",
    1: "Disgust",
    2: "Fear",
    3: "Happy",
    4: "Neutral",
    5: "Sad",
    6: "Surprise"
}

# Function to preprocess an image
def preprocess_image(image_path):
    # Load the image in grayscale
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    
    # Resize the image to 48x48 pixels
    img = cv2.resize(img, (48, 48))
    
    # Normalize pixel values (scale between 0 and 1)
    img = img / 255.0
    
    # Reshape to add batch dimension (1 image, 48x48 pixels, 1 color channel)
    img = np.reshape(img, (1, 48, 48, 1))
    
    return img

# Path to the test image
image_path = r"C:\Users\HP\SAD.jpg"  # Corrected image path

# Preprocess the image
processed_image = preprocess_image(image_path)

# Make a prediction using the model
prediction = model.predict(processed_image)

# Get the predicted class index
predicted_class = np.argmax(prediction)

# Get the emotion label corresponding to the predicted class
predicted_emotion = EMOTION_LABELS[predicted_class]

# Print the result
print(f"Predicted Emotion: {predicted_emotion}")



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step
Predicted Emotion: Surprise


In [7]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# Load dataset
df = pd.read_csv("fer2013_combined.csv")

# Convert pixel strings to numpy arrays
X = np.array([np.fromstring(pixels, dtype=np.uint8, sep=' ').reshape(48, 48, 1) for pixels in df["pixels"]])
y = to_categorical(df["emotion"], num_classes=7)

# Normalize pixel values
X = X / 255.0

# Split into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True)
datagen.fit(X_train)

# improved CNN model
model = Sequential([
    Conv2D(64, (3,3), activation='relu', input_shape=(48,48,1)),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(0.25),
    
    Conv2D(128, (3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(0.25),
    
    Conv2D(256, (3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(0.25),
    
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(7, activation='softmax')
])

# Compile model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='categorical_crossentropy', metrics=['accuracy'])

# Learning rate scheduler
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train model
model.fit(datagen.flow(X_train, y_train, batch_size=64),
          validation_data=(X_val, y_val),
          epochs=30,
          callbacks=[lr_scheduler, early_stopping])

# Save model
model.save("improved_emotion_model.h5")

# Evaluate model
val_loss, val_acc = model.evaluate(X_val, y_val)
print(f"Validation Accuracy: {val_acc * 100:.2f}%")


C:\Users\HP\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\HP\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
449/449 ━━━━━━━━━━━━━━━━━━━━ 303s 625ms/step - accuracy: 0.2274 - loss: 2.2605 - val_accuracy: 0.2621 - val_loss: 1.9755 - learning_rate: 0.0010
Epoch 2/30
449/449 ━━━━━━━━━━━━━━━━━━━━ 267s 594ms/step - accuracy: 0.2999 - loss: 1.7182 - val_accuracy: 0.3344 - val_loss: 1.6625 - learning_rate: 0.0010
Epoch 3/30
449/449 ━━━━━━━━━━━━━━━━━━━━ 275s 613ms/step - accuracy: 0.3474 - loss: 1.6394 - val_accuracy: 0.4122 - val_loss: 1.5237 - learning_rate: 0.0010
Epoch 4/30
449/449 ━━━━━━━━━━━━━━━━━━━━ 265s 591ms/step - accuracy: 0.3861 - loss: 1.5593 - val_accuracy: 0.3656 - val_loss: 1.6195 - learning_rate: 0.0010
Epoch 5/30
449/449 ━━━━━━━━━━━━━━━━━━━━ 269s 599ms/step - accuracy: 0.4008 - loss: 1.5267 - val_accuracy: 0.4072 - val_loss: 1.4782 - learning_rate: 0.0010
Epoch 6/30
449/449 ━━━━━━━━━━━━━━━━━━━━ 313s 580ms/step - accuracy: 0.4089 - loss: 1.5007 - val_accuracy: 0.4599 - val_loss: 1.4317 - learning_rate: 0.0010
Epoch 7/30
449/449 ━━━━━━━━━━━━━━━━━━━━ 239s 532ms/step - accura

225/225 ━━━━━━━━━━━━━━━━━━━━ 10s 43ms/step - accuracy: 0.5907 - loss: 1.0994
Validation Accuracy: 59.50%


In [16]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam

# Load the trained model
model = load_model("emotion_detection_model.h5")

# Recompile the model (optional but recommended to avoid the warning)
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Define emotion labels (to map the predictions to human-readable emotions)
EMOTION_LABELS = {
    0: "Angry",
    1: "Disgust",
    2: "Fear",
    3: "Happy",
    4: "Neutral",
    5: "Sad",
    6: "Surprise"
}

# Function to preprocess an image
def preprocess_image(image_path):
    # Load the image in grayscale
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    
    # Resize the image to 48x48 pixels
    img = cv2.resize(img, (48, 48))
    
    # Normalize pixel values (scale between 0 and 1)
    img = img / 255.0
    
    # Reshape to add batch dimension (1 image, 48x48 pixels, 1 color channel)
    img = np.reshape(img, (1, 48, 48, 1))
    
    return img

# Path to the test image
image_path = r"C:\Users\HP\images.jpg"  # Corrected image path

# Preprocess the image
processed_image = preprocess_image(image_path)

# Make a prediction using the model
prediction = model.predict(processed_image)

# Get the predicted class index
predicted_class = np.argmax(prediction)

# Get the emotion label corresponding to the predicted class
predicted_emotion = EMOTION_LABELS[predicted_class]

# Print the result
print(f"Predicted Emotion: {predicted_emotion}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 168ms/step
Predicted Emotion: Fear
